### 介绍TensorFlow自定义训练模型
自定义模型可以帮助我们理解深度学习，但是还是Keras模型封装使用方便

In [1]:
import tensorflow as tf

### 自动求导tf.GradentTape

- **GradientTape**是eager模式下计算梯度用的

- **watch(tensor)**

  作用：确保某个tensor被tape追踪 

  参数:tensor: 一个Tensor或者一个Tensor列表

- **gradient(target, sources)**

  作用：根据tape上面的上下文来计算某个或者某些tensor的梯度参数

  target: 被微分的Tensor或者Tensor列表，你可以理解为经过某个函数之后的值

  sources: Tensors 或者Variables列表（当然可以只有一个值）. 你可以理解为函数的某个变量

  返回:
  
  一个列表表示各个变量的梯度值，和source中的变量列表一一对应，表明这个变量的梯度。
  
  上面的例子中的梯度计算部分可以更直观的理解这个函数的用法。


In [2]:
#求y=x^2在x=3处的导数
x=tf.constant(3.0)

with tf.GradientTape() as g:
    g.watch(x)
    y=x*x#把要求导的函数放在with块里面
    
dy_dx=g.gradient(y,x)

tf.print(dy_dx)

6


## 自定义模型
包括但是不限于：
- 构建基本的模型结构（构建前向传播、定义损失函数、定义优化函数、求导、得到预测值、后向传播，更新参数）
- 构建多层循环(epoch)、多层数据（batch）
- 构建评估函数


In [3]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu') #隐藏层
        self.dense_2 = tf.keras.layers.Dense(num_classes)#输出层
    
    @tf.function(input_signature=[tf.TensorSpec([None,32],tf.float32)])
    def call(self, inputs):
        #定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [4]:
# #下面是前向传播与后向传播的一个完整过程，详细解释
# model = MyModel(num_classes=10)

# #损失函数
# loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# #优化器
# optimizer = tf.keras.optimizers.Adam()

# #梯度
# with tf.GradientTape() as tape:
#     predictions = model(data)
#     loss = loss_object(labels, predictions)
    
# gradients = tape.gradient(loss, model.trainable_variables) #求梯度

# optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # 把计算出来的梯度更新到变量上去

上述是前向传播与后向传播的一个完整过程；然后将之作为TensorFlow模型的迭代循环的一个迭代
- 每个Batch循环，然后迭代更新
- 每个epoch循环，然后迭代更新
另外，还需要添加评估函数

In [5]:
#10分类问题
import numpy as np
x_train = np.random.random((1000, 32))
y_train = np.random.random((1000, 10))
x_val = np.random.random((200, 32))
y_val = np.random.random((200, 10))
x_test = np.random.random((200, 32))
y_test = np.random.random((200, 10))

# 优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# 损失函数
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# 准备metrics函数
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# 准备训练数据集
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# 准备测试数据集
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [6]:
model = MyModel(num_classes=10)
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # 遍历数据集的batch_size
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):        
        
        #一个batch
        with tf.GradientTape() as tape:
            predicts = model(x_batch_train)
            loss_value = loss_fn(y_batch_train, predicts)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))####

        # 更新训练集的metrics
        train_acc_metric(y_batch_train, predicts)     
            
            
    # 在每个epoch结束时显示metrics。
    train_acc = train_acc_metric.result()
    print('Training acc over epoch: %s' % (float(train_acc),))
    # 在每个epoch结束时重置训练指标
    train_acc_metric.reset_states()#!!!!!!!!!!!!!!!

    # 在每个epoch结束时运行一个验证集。
    for x_batch_val, y_batch_val in val_dataset:
        val_predicts = model(x_batch_val)
        # 更新验证集merics
        val_acc_metric(y_batch_val, val_predicts)
    val_acc = val_acc_metric.result()
    print('Validation acc: %s' % (float(val_acc),))
    val_acc_metric.reset_states()
    
    #显示测试集

Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training acc over epoch: 0.10199999809265137
Validation acc: 0.11999999731779099
Start of epoch 1
Training acc over epoch: 0.11100000143051147
Validation acc: 0.10999999940395355
Start of epoch 2
Training acc over epoch: 0.10999999940395355
Validation acc: 0.11500000208616257


In [7]:
## 保存模型
tf.saved_model.save(model,"auto_defined_model_v1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: auto_defined_model_v1\assets


## 模型保存，有两种方式

### 简单的模型保存、加载、预测(程序报错，不知道原因在哪里，哪里应该定义inputs)

In [12]:
model.save('defined_model_tf',save_format='tf')

ValueError: Model <__main__.MyModel object at 0x0000023950CE90C8> cannot be saved because the input shapes have not been set. Usually, input shapes are automatically determined from calling .fit() or .predict(). To manually set the shapes, call model._set_inputs(inputs).

In [13]:
new_model = tf.keras.models.load_model('defined_model_tf')
new_model.predict(x_test)

OSError: SavedModel file does not exist at: defined_model_tf/{saved_model.pbtxt|saved_model.pb}

### 模型保存+部署+预测

In [15]:
tf.saved_model.save(model,'defined_model_tf_model_v4')


INFO:tensorflow:Assets written to: defined_model_tf_model_v4\assets


In [20]:
restored_saved_model = tf.saved_model.load('defined_model_tf_model_v4')
!saved_model_cli show --dir defined_model_tf_model_v4 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [17]:
f = restored_saved_model.signatures["serving_default"]

In [19]:
f(inputs = tf.constant(x_test.tolist()))

{'output_0': <tf.Tensor: id=6290, shape=(200, 10), dtype=float32, numpy=
 array([[ 0.4305519 ,  0.3081314 , -0.64959145, ...,  0.26828435,
          0.51096547,  0.36144125],
        [ 0.8197312 ,  0.11326784, -0.20322207, ...,  0.098868  ,
          0.6067729 ,  0.18710305],
        [ 0.43130603,  0.714454  , -0.7050206 , ..., -0.20296393,
          1.0893133 ,  0.2883138 ],
        ...,
        [ 0.4604233 ,  0.3721704 , -0.35606158, ...,  0.05747841,
          0.5738932 ,  0.5697481 ],
        [ 0.32909262,  0.4179882 , -0.29683197, ..., -0.26710546,
          0.45765913,  1.0516205 ],
        [ 0.4513636 ,  0.63266367, -0.32223296, ..., -0.04709849,
          0.4686654 ,  0.66623527]], dtype=float32)>}